In [161]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv

from pymongo import MongoClient
import pandas as pd


import src.haversine_function as distance
import src.nearby_function as nearby



In [219]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("Crunchbase")

In [220]:
# filter with regex "desing", 'total_money_raised', "offices[0].country_code"

In [282]:
#DESIGN or tech companies nearby
filter_10 = {"description": {"$regex": "design"}}
filter_11 = {"description": {"$regex": "technology"}}
filter_12 = {"description": {"$regex": "software"}}

filter_2 = {"tag_list": {"$regex": "design"}}

filter_3 = {"name": {"$regex": "design"}}

filter_40 = {"category_code": {"$regex": "design"}}
filter_41 = {"category_code": {"$regex": "games_video"}}
filter_42 = {"category_code": {"$regex": "web"}}
filter_43 = {"category_code": {"$regex": "mobile"}}
filter_44 = {"category_code": {"$regex": "photo_video"}}

#Near to SUCCESSFUL tech startups
filter_50 = {"total_money_raised": {"$regex": "M"}}
filter_51 = {"total_money_raised": {"$regex": "B"}}
filter_52 = {"total_money_raised": {"$regex": "^k"}}
filter_53 = {"total_money_raised":{ "$not": {"$regex": "/$0/"}}}




all_conditions = [filter_10, filter_11, filter_12, filter_2, filter_3, filter_40, 
                  filter_41, filter_42, filter_43, filter_44, filter_50, filter_51, filter_52, filter_53]

In [283]:
projection = {"name": 1, 
              "number_of_employees": 1,
              "total_money_raised": 1,
              "offices.country_code": 1,
              "offices.city": 1,
              "offices.latitude": 1,
              "offices.longitude": 1,
              "_id":0}

result = list(c.find({"$or": all_conditions,
                      
                "number_of_employees": {"$lte": 150 } # STARTUP
                     
                     }, projection))

df = pd.DataFrame(result)

SyntaxError: ':' expected after dictionary key (628422675.py, line 10)

In [223]:
df

,name,number_of_employees,total_money_raised,offices
0,Wetpaint,47,$39.8M,"[{'city': 'Seattle', 'country_code': 'USA', 'l..."
1,Digg,60,$45M,"[{'city': 'San Francisco', 'country_code': 'US..."
2,Geni,18,$16.5M,"[{'city': 'West Hollywood', 'country_code': 'U..."
3,Fox Interactive Media,0,$0,"[{'city': 'Beverly Hills', 'country_code': 'US..."
4,Scribd,50,$25.8M,"[{'city': 'San Francisco', 'country_code': 'US..."
...,...,...,...,...
8130,OfficialVirtualDJ,102,€25M,"[{'city': 'London', 'country_code': 'GBR', 'la..."
8131,Eazeeloans,25,$0,"[{'city': 'Mumbai', 'country_code': 'IND', 'la..."
8132,DocASAP,7,$800k,"[{'city': 'New York', 'country_code': 'USA', '..."
8133,cotralis,1,$0,[]


In [224]:
col_to_extract = df['offices']
col_to_extract[0][0]

{'city': 'Seattle',
 'country_code': 'USA',
 'latitude': 47.603122,
 'longitude': -122.333253}

In [225]:
def extraction(col, new_col, data):    
    list_ = []
    for i in range(len(col)):
        try:           
            list_.append(col[i][0][data])             
        except:
            list_.append("null")
    df[new_col]= list_append
    return

In [226]:
get_info(col_to_extract, "Country", "country_code")
get_info(col_to_extract, "City", "city")
get_info(col_to_extract, "lat", "latitude")
get_info(col_to_extract, "lon", "longitude")

In [227]:
df

,name,number_of_employees,total_money_raised,offices,Country,City,lat,lon
0,Wetpaint,47,$39.8M,"[{'city': 'Seattle', 'country_code': 'USA', 'l...",USA,Seattle,47.603122,-122.333253
1,Digg,60,$45M,"[{'city': 'San Francisco', 'country_code': 'US...",USA,San Francisco,37.764726,-122.394523
2,Geni,18,$16.5M,"[{'city': 'West Hollywood', 'country_code': 'U...",USA,West Hollywood,34.090368,-118.393064
3,Fox Interactive Media,0,$0,"[{'city': 'Beverly Hills', 'country_code': 'US...",USA,Beverly Hills,34.076179,-118.39417
4,Scribd,50,$25.8M,"[{'city': 'San Francisco', 'country_code': 'US...",USA,San Francisco,37.789634,-122.404052
...,...,...,...,...,...,...,...,...
8130,OfficialVirtualDJ,102,€25M,"[{'city': 'London', 'country_code': 'GBR', 'la...",GBR,London,None,None
8131,Eazeeloans,25,$0,"[{'city': 'Mumbai', 'country_code': 'IND', 'la...",IND,Mumbai,None,None
8132,DocASAP,7,$800k,"[{'city': 'New York', 'country_code': 'USA', '...",USA,New York,None,None
8133,cotralis,1,$0,[],null,null,null,null


In [228]:
df.drop(['offices'],axis =1, inplace=True)
df.dropna(inplace=True)
df = df[df["Country"] != "null"]

In [155]:
df

,name,number_of_employees,total_money_raised,Country,City,lat,lon
0,Wetpaint,47,$39.8M,USA,Seattle,47.603122,-122.333253
1,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
2,Geni,18,$16.5M,USA,West Hollywood,34.090368,-118.393064
3,Fox Interactive Media,0,$0,USA,Beverly Hills,34.076179,-118.39417
4,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
...,...,...,...,...,...,...,...
4461,Tiny Speck,45,$17.2M,CAN,Vancouver,49.282455,-123.109217
4462,My TV Shows,2,$0,PRT,Lisbon,38.752367,-9.146221
4465,Atlantic BT,30,$0,USA,Raleigh,35.896906,-78.680678
4469,acrossair,6,$0,GBR,London,51.517742,-0.116537


In [229]:
df["Country"].value_counts()[:1]

USA    3437
Name: Country, dtype: int64

In [230]:
df['City'].where(df['Country'] == 'USA').value_counts()[:3]

San Francisco    320
New York         249
Seattle           89
Name: City, dtype: int64

In [231]:
df_SanF = df[df['City'] == 'San Francisco']
df_SanF

,name,number_of_employees,total_money_raised,Country,City,lat,lon
1,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
4,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
8,Powerset,60,$22.5M,USA,San Francisco,37.778613,-122.395289
9,Technorati,35,$32.1M,USA,San Francisco,37.779558,-122.393041
12,Kyte,40,$23.4M,USA,San Francisco,37.788482,-122.409173
...,...,...,...,...,...,...,...
7776,UTOPY,40,$7.5M,USA,San Francisco,37.767575,-122.411157
7821,ZoomSystems,100,$87M,USA,San Francisco,37.781689,-122.391061
7879,Mondo Media,9,$0,USA,San Francisco,37.764169,-122.402021
7902,TwtBks,3,$25k,USA,San Francisco,37.761102,-122.416001


In [ ]:
df

In [ ]:
'''STARTING THE PROJECT

:uno: Explore database & filter it

:dos: Explore how to query (no matter city yet):

Create small function to query starbucks (limit=1, city=whichever)
Small function to query something else (limit=1, city=whichever)
Maybe these two can be the same and it’s just a matter of parametrization?
and so on :cara_ligeramente_sonriente:
:tres: Focus on getting one thing at a time: you’ll scale things up later on
It’s about making decisions. Try to justify them with data as much as you can, but it’s okay as some of them you’ll have to make a bit randomly.
Goal of the project is:
Querying a database
Extracting information through an API
Loading that information back into a database
Aggregate and explore by: doing geoqueries
Visualize using maps :mapamundi:'''

In [125]:
data = {'Requirements ':   ['Design companies nearby', 
                            '30% of staff have childs /parks and schools nearby/,
                            'Near successful tech startups',
                            'Starbucks not too far',
                            'Account managers need to travel a lot',
                            'Place to go party',
                            'The CEO is vegan',
                            'A basketball stadium must be around 10 Km',
                            'Dog hairdresser not too far'],
        
        'Priorization Factor': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
  


df_info = pd.DataFrame(data)
df_info                      

,Requirements,Priorization Factor
0,Design companies nearby,1
1,30% of staff have childs,1
2,Near successful tech startups,1
3,Starbucks not too far,1
4,Account managers need to travel a lot,1
5,Place to go party,1
6,The CEO is vegan,1
7,A basketball stadium must be around 10 Km,1
8,Dog hairdresser not too far,1


# QUERY = Near to Design Companies

In [274]:
def type_point (list_):
    return {"type": "Point", "coordinates": list_}

In [275]:
def get_closeby_things (location, max_distance):
    
    proy_ = {"_id": 0}

    converted = type_point (location)
    query = {"$or": conditions, "location": {
        "$near": {
            "$geometry": converted, "$maxDistance": max_distance
        }}

    }

    return list(c.find(query, proy_))

In [276]:
location11=[-122.333253, 47.603122]

In [277]:
max_distance = 2000

In [278]:
filter_40 = {"category_code": {"$regex": "design"}}
filter_13 = {"description": {"$regex": "design"}}
filter_2 = {"tag_list": {"$regex": "design"}}

conditions = [filter_40, filter_13, filter_2]

In [279]:
result_design= get_closeby_things (location11, max_distance)


OperationFailure: error processing query: ns=ironhack.CrunchbaseTree: $and
    $or
        category_code regex /design/
        description regex /design/
        tag_list regex /design/
    GEONEAR  field=location maxdist=2000 isNearSphere=0
Sort: {}
Proj: { _id: 0 }
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=ironhack.CrunchbaseTree: $and\n    $or\n        category_code regex /design/\n        description regex /design/\n        tag_list regex /design/\n    GEONEAR  field=location maxdist=2000 isNearSphere=0\nSort: {}\nProj: { _id: 0 }\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [162]:
load_dotenv()

True

In [163]:
token_for_foursquare = os.getenv("API_Key")

In [180]:
def get_results_from_foursquare (query, location, limit=1):
    
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_for_foursquare,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

# QUERY = PARK

In [236]:
query_10 = "park"

In [237]:
location=[47.603122, -122.333253]

In [238]:
response_park = get_results_from_foursquare(query_10,location)

In [239]:
latitude = response_park["results"][0]["geocodes"]["main"]["latitude"]
longitude = response_park["results"][0]["geocodes"]["main"]["longitude"]
address = response_park["results"][0]["location"]["address"]
name = response_park["results"][0]["name"]
distance = response_park["results"][0]['distance']


print(distance)
print(coord1)
print(address)
print(name)

1868
[47.592823, -122.316914]
1635 11th Ave W
Cal Anderson Park


In [ ]:
# Do the same thing: limit=50

all_parks = get_results_from_foursquare (query_10, location, limit=50)

In [ ]:
new_list_park = []
for i in all_parks["results"]:
    
    name = i["name"]
    address =  i["location"]["formatted_address"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_park.append({"name":name, "lat":lat, "lon":lon, "type":type_})
    
new_list_park[:1]

# QUERY = Schools

In [240]:
query_11 = "school"

In [187]:
location=[47.603122, -122.333253]

In [241]:
response_school = get_results_from_foursquare(query_11,location)

In [242]:
latitude = response_school["results"][0]["geocodes"]["main"]["latitude"]
longitude = response_school["results"][0]["geocodes"]["main"]["longitude"]
address = response_school["results"][0]["location"]["address"]
name = response_school["results"][0]["name"]
distance = response_school["results"][0]['distance']


print(distance)
print(coord1)
print(address)
print(name)

4672
[47.592823, -122.316914]
2619 4th Ave N
Seattle Country Day School


# QUERY = Airports

In [291]:
query_50 = "Airports"

In [292]:
location=[47.9081441,-122.3514318]

In [295]:
response_park = get_results_from_foursquare(query_50,location)
response_park

{'results': [{'fsq_id': '453bfd7ef964a520083c1fe3',
   'categories': [{'id': 19031,
     'name': 'Airport',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 5473,
   'geocodes': {'main': {'latitude': 47.907874, 'longitude': -122.277689},
    'roof': {'latitude': 47.907874, 'longitude': -122.277689}},
   'link': '/v3/places/453bfd7ef964a520083c1fe3',
   'location': {'address': '3220 100th St SW',
    'address_extended': 'Ste A',
    'census_block': '530610419011006',
    'country': 'US',
    'dma': 'Seattle-Tacoma',
    'formatted_address': '3220 100th St SW, Everett, WA 98204',
    'locality': 'Everett',
    'postcode': '98204',
    'region': 'WA'},
   'name': 'Snohomish County Airport (PAE)',
   'related_places': {'children': [{'fsq_id': '4e5177322271a1bdc3fb027c',
      'name': 'Air Traffic Control Tower'},
     {'fsq_id': '518e941e498e540694ef6bda', 'name': 'Helicopters Northwest'},
     {'fs

In [294]:
latitude = response_park["results"][0]["geocodes"]["main"]["latitude"]
longitude = response_park["results"][0]["geocodes"]["main"]["longitude"]
address = response_park["results"][0]["location"]["address"]
name = response_park["results"][0]["name"]
distance = response_park["results"][0]['distance']


print(distance)
print(coord1)
print(address)
print(name)

5473
[47.592823, -122.316914]
3220 100th St SW
Snohomish County Airport (PAE)


# QUERY = Schools

In [240]:
query_11 = "school"

In [187]:
location=[47.603122, -122.333253]

In [241]:
response_park = get_results_from_foursquare(query_11,location)

In [242]:
latitude = response_park["results"][0]["geocodes"]["main"]["latitude"]
longitude = response_park["results"][0]["geocodes"]["main"]["longitude"]
address = response_park["results"][0]["location"]["address"]
name = response_park["results"][0]["name"]
distance = response_park["results"][0]['distance']


print(distance)
print(coord1)
print(address)
print(name)

4672
[47.592823, -122.316914]
2619 4th Ave N
Seattle Country Day School


In [ ]:
response_function["results"][0]